# Overview of the current state, features and requiremnents of `f3dasm`
*Meeting 24/03/2022*

Contents of this meeting 
- Requirements of first release of `f3dasm`
- Installation and modularity
- Use cases (not in this jupyter notebook)

## Requirements of first release of f3dasm

> Distinction between python package and repository
> - Python package contains the code that is used when installing the package (`pip install f3dasm`) as a user. It contains only the `main` branch version
> - The repository is mainly for developers and to showcase studies (more on that later)

### System requirements
The first release of `f3dasm` **should**:
1. comply to the Bessa Group code of conduct
2. compatible with Python 3.7 to 3.10
3. compatible with the three major operations system (Linux, MacOS, Ubuntu)
4. available as a `pip` install

The first release of `f3dasm` **should not**:
1. use `conda` packages as requirements
2. be hosted on other package distribution platforms (`poetry`, `conda`)

### Feature requirements
The first release of `f3dasm` should:
1. have a way to parametrize a **design-of-experiments** from their own program
2. give the user the option to **sample** and **optimize** from their design with various optimizers
3. give the user guidance in **parallelizing** and ordering their experiment-data
4. provide high-level classes to deploy **simulations**.
4. give the user ways of deploying their **experiment** at the HPC (TORQUE system)

The first release of `f3dasm` **should not**:
1. force the user to use the built-in machine learning models and optimizers (these are extensions)
2. include state-of-the-art machine learning models as extensions

## Installation and modularity

`f3dasm` can be installed in two ways:

### Installing from `pip`

The package is split up into three parts:
- the **core** package: contains the minimal installation to use `f3dasm` without extended features. Installed with `pip install f3dasm`
- the **extensions**: for each of the blocks, extensions can be installed to extend the choice of implementations. Installed with `pip install f3dasm[extension]`
- the **developement** package: contains the full installation plus requirements for developing on `f3dasm`. Installed with `pip install f3dasm[dev]`

The following extensions are available:
- **machinelearning**: containing various `tensorflow` related models
- **sampling**: containing sampling strategies from `SALib`
- **optimization**: containing various optimizers from `GPyOpt`, `pygmo` and `tensorflow`

### Installing from source

Advised for developers only. 
- Make sure you have the build dependencies (`setuptools`, `wheel`). Or install the development requirements (`pip install -r requirements_dev.txt`)
- Clone the repository and install the package in editable mode (`pip install -e .[dev]`)

In [1]:
import f3dasm

2023-03-24 14:07:32.396337: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 14:07:32.594329: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-24 14:07:33.411381: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.1/lib64
2023-03-24 14:07:33.411465: W tensorflow/compiler/xla/stream

We can show our exact installation with the following function:

In [2]:
f3dasm.show_versions()


f3dasm:
       f3dasm: 0.2.98

System:
    python: 3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:49:35)  [GCC 10.4.0]
executable: /home/martin/miniconda3/envs/f3dasm_env3/bin/python
   machine: Linux-5.14.0-1058-oem-x86_64-with-glibc2.10

Core package dependencies:
        numpy: 1.23.5
        scipy: 1.9.3
       pandas: 1.5.2
   matplotlib: 3.6.2
       pathos: 0.3.0
        hydra: 1.2.0
     autograd: No __version__ attribute!

Machine learning extension:
   tensorflow: 2.11.0

Optimization extension:
       GPyOpt: 1.2.6
          GPy: 1.10.0
   tensorflow: 2.11.0
        pygmo: 2.19.0

Sampling extension:
        SALib: None


### Namespace

Base classes are available from the global namespace:

In [3]:
f3dasm.Sampler, f3dasm.Optimizer, f3dasm.Model

(f3dasm.sampling.sampler.Sampler,
 f3dasm.optimization.optimizer.Optimizer,
 f3dasm.machinelearning.model.Model)

The design space features are also available from the global:

In [4]:
f3dasm.DesignSpace, f3dasm.ExperimentData, f3dasm.ContinuousParameter, f3dasm.make_nd_continuous_design

(f3dasm.design.design.DesignSpace,
 f3dasm.design.experimentdata.ExperimentData,
 f3dasm.design.parameter.ContinuousParameter,
 <function f3dasm.design.design.make_nd_continuous_design(bounds: numpy.ndarray, dimensionality: int) -> f3dasm.design.design.DesignSpace>)

Other utility functions that are under the global namespace:

In [5]:
f3dasm.run_optimization, f3dasm.show_versions

(<function f3dasm.run_optimization.run_optimization(optimizer: f3dasm.optimization.optimizer.Optimizer, function: f3dasm.functions.function.Function, sampler: f3dasm.sampling.sampler.Sampler, iterations: int, seed: int, number_of_samples: int = 30) -> f3dasm.design.experimentdata.ExperimentData>,
 <function f3dasm._show_versions.show_versions()>)

All implementations can found under the respective submodules:
- `f3dasm.sampling`
- `f3dasm.machinelearning`
- `f3dasm.optimization`
- `f3dasm.simulation`

In [6]:
f3dasm.optimization.Adam, f3dasm.sampling.RandomUniform, f3dasm.machinelearning.LinearRegression

(f3dasm.optimization.adam.Adam,
 f3dasm.sampling.randomuniform.RandomUniform,
 f3dasm.machinelearning.linear_regression.LinearRegression)

Furthermore, there two more submodules:
- `f3dasm.functions`: for benchmarkfunctions, they can be used in any stage in the process
- `f3dasm.experiment`: Experiment utilities, for datacollection, paralellization and HPC support

## Modularity

Whenever you import the f3dasm package, the implementations are only loaded when you create an instance of the class.
Only then it is checked if a particular dependency is installed or not. This keeps the package lightweight and customizable

Example, The `SALib` library is not installed. I can load the `LatinHyperCube` sampler without any problem.

In [7]:
f3dasm.sampling.LatinHypercube

f3dasm.sampling.latinhypercube.LatinHypercube

Only when I create an instance of this sampler, an ImportError is thrown:

In [8]:
sampler = f3dasm.sampling.LatinHypercube(design=0)

ImportError: Tried to import 'SALib' but failed. Please make sure that you have installed the extension 'sampling' correctly to use this feature. Actual error: No module named 'SALib'.

All the available samplers within your installation can be shown with `f3dasm.sampling.SAMPLERS`

> same logic goes for `OPTIMIZERS` and `MODELS`

In [ ]:
f3dasm.sampling.SAMPLERS

## Other notes

- way of working with f3dasm: 'fully f3dasm' or 'core f3dasm' (base classes both protocol and parent class)
- lacking implementation of simulator
- enthousiasm from bessa group
- after release v.1.0